In [2]:
# IMPORTAÇÃO DA LIBS PARA ETL 
import basedosdados as bd
import pandas as pd 
import numpy as np 

In [3]:
## IMPORTAÇÃO PARA A ANALISE GRAFICA DOS DADOS
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import seaborn as sns
from pandas_profiling import ProfileReport
import folium 
import seaborn as sbn
from folium.plugins import HeatMap
from folium import plugins

In [4]:
df = bd.read_sql(
    '''
    SELECT chmd.id_chamado, chmd.data_inicio, chmd.situacao,chmd.id_bairro, chmd.tipo, chmd.subtipo, br.nome as bairro, br.subprefeitura, chmd.latitude,chmd.longitude
    FROM datario.administracao_servicos_publicos.chamado_1746 chmd 
    LEFT JOIN datario.dados_mestres.bairro br ON br.id_bairro = chmd.id_bairro
    WHERE DATE(data_inicio) = DATE('2023-04-01');
    ''',
    billing_project_id='case-escritorio-de-dados'
)

Downloading: 100%|██████████| 73/73 [00:00<00:00, 208.27rows/s]


In [5]:
df.head(1)

,id_chamado,data_inicio,situacao,id_bairro,tipo,subtipo,bairro,subprefeitura,latitude,longitude
0,18516246,2023-04-01 00:55:38,Encerrado,None,Ônibus,Verificação de ar condicionado inoperante no ô...,None,None,NaN,NaN


### 1) Quantos chamados foram abertos no dia 01/04/2023?

In [7]:
print('Os numeros de chamados abertos no dia 01-04-2023 foram {} casos'.format(df['id_chamado'].count()))

Os numeros de chamados abertos no dia 01-04-2023 foram 73 casos


### 2) Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [8]:
com_mais_chamadas = df.groupby('tipo').size().sort_values(ascending=False).reset_index(name='quantity')

print(com_mais_chamadas.head(5))

                       tipo  quantity
0           Poluição sonora        24
1  Estacionamento Irregular         9
2        Iluminação Pública         8
3          Remoção Gratuita         5
4                   Limpeza         5


In [13]:
fig = px.histogram(df, x="tipo", barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 3) Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [9]:
bairro_com_mais_chamados = df.groupby('bairro')['tipo'].size().sort_values(ascending=False).reset_index(name='quantity')

print(bairro_com_mais_chamados.head(3))

              bairro  quantity
0  Engenho de Dentro         8
1       Campo Grande         6
2             Leblon         6


In [15]:
fig = px.histogram(bairro_com_mais_chamados, x="bairro", y='quantity', barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 4) Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [10]:
maioresSubprefeituras = df.groupby('subprefeitura')['tipo'].size().sort_values(ascending=False).reset_index(name='quantity')

print(maioresSubprefeituras.head(5))

     subprefeitura  quantity
0       Zona Norte        25
1         Zona Sul        12
2       Zona Oeste        10
3           Centro         7
4  Barra da Tijuca         6


In [14]:
fig = px.histogram(df, x="subprefeitura", barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 5) Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?
- como se tratava de um problema de ar condicionado em um onibus, acredito que localização exata em si não ficou registrada

In [11]:
df[df['id_bairro'].isna()]

,id_chamado,data_inicio,situacao,id_bairro,tipo,subtipo,bairro,subprefeitura,latitude,longitude
0,18516246,2023-04-01 00:55:38,Encerrado,None,Ônibus,Verificação de ar condicionado inoperante no ô...,None,None,NaN,NaN


### (EXTRA) Distribuição dos pontos de Chamada no dia 01/04/2023

In [22]:
mapa = folium.Map(location=[-22.9028, -43.2078],zoom_start=12.4)

In [23]:
# Estações iniciais 

df_lat_long_start = df[['latitude', 'longitude']].dropna()

dataStart = df_lat_long_start.values.tolist()
 
hmStart = HeatMap(dataStart,gradient={0.1: 'blue', 0.3: 'lime', 0.5: 'yellow', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.05, 
                max_opacity=0.9, 
                radius=25,
                use_local_extrema=False).add_to(mapa)

In [28]:
Bairros = df.groupby(['bairro']).agg({
    'tipo' :'first',
    'bairro':'first',
    'latitude':'first', 
    'longitude':'first',
    'subtipo':'count'
    }).sort_values(by='subtipo',ascending=False)

In [32]:
chamados_bairros = Bairros.dropna()
# MAKER POP'UPS DAS ESTAÇÕES
for _, r in chamados_bairros.iterrows():
    lat = r['latitude']
    lon = r['longitude']
    folium.Marker(location=[lat, lon],
                  popup=['{} <br> '.format(r['bairro']), '{} <br>'.format(r['subtipo'])]).add_to(mapa)

mapa

### Extração dos dados com BigQuery e a Lib BasedosDados

### 6) Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [54]:
df_query= bd.read_sql(
    '''
    SELECT chmd.id_chamado, chmd.tipo, chmd.subtipo, chmd.data_inicio
    FROM datario.administracao_servicos_publicos.chamado_1746 chmd
    WHERE chmd.subtipo = 'Perturbação do sossego' AND chmd.data_inicio BETWEEN '2022-01-01' AND '2023-12-31';
    ''',
    billing_project_id='case-escritorio-de-dados'
)

Downloading: 100%|██████████| 42408/42408 [00:03<00:00, 11193.87rows/s]


In [69]:
print('Os numeros de chamados abertos com o subtipo "Perturbação do sossego" desde 01/01/2022 até 31/12/2023 foram de {} casos'.format(df_query['id_chamado'].count()))

Os numeros de chamados abertos com o subtipo "Perturbação do sossego" desde 01/01/2022 até 31/12/2023 foram de 42408 casos


### 7) Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [39]:
df2 = bd.read_sql(
    '''
    SELECT chmd.id_chamado, chmd.tipo, chmd.subtipo, htl.evento, chmd.data_inicio
    FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos htl
    INNER JOIN datario.administracao_servicos_publicos.chamado_1746 chmd ON DATE(chmd.data_inicio) >= DATE(htl.data_inicial) AND DATE(chmd.data_inicio) <= DATE(htl.data_final) 
    WHERE chmd.subtipo = 'Perturbação do sossego' AND chmd.data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
    ''',
    billing_project_id='case-escritorio-de-dados'
)

Downloading: 100%|██████████| 1212/1212 [00:00<00:00, 2796.95rows/s]


### 8) Quantos chamados desse subtipo foram abertos em cada evento?

In [66]:
numero_chamados = df2.groupby('evento')['subtipo'].size().sort_values(ascending=False).reset_index(name='quantity')
print(numero_chamados)

        evento  quantity
0  Rock in Rio       834
1     Carnaval       241
2    Reveillon       137


In [42]:
fig = px.histogram(numero_chamados, x="evento", y='quantity', barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 9) Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [70]:
media = round(df2.groupby(['evento', pd.Grouper(key='data_inicio', freq='D')]).size().groupby('evento').mean(),2)
print(media)

evento
Carnaval        60.25
Reveillon       45.67
Rock in Rio    119.14
dtype: float64


In [51]:
fig = px.histogram(numero_chamados, x="evento", y= media, barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 10) Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [61]:
media_Total = round(len(df_query) / ((pd.to_datetime('2023-12-31') - pd.to_datetime('2022-01-01')).days),2)

In [71]:
tabela = pd.DataFrame({
    'Média dos Eventos': media,
    'Média do período (22/23)': media_Total
})

print(tabela)

             Média dos Eventos  Média do período (22/23)
evento                                                  
Carnaval                 60.25                     58.17
Reveillon                45.67                     58.17
Rock in Rio             119.14                     58.17
